# Notebook para o sistema de recomendação baseado na similaridade do cosseno

Este notebook apresenta a implementação de um sistema de recomendação nutricional que utiliza a similaridade entre perfis nutricionais de alimentos da Tabela INSA. O objetivo é sugerir alimentos com composição nutricional semelhante ao alimento de interesse, facilitando escolhas alimentares informadas e equilibradas.

Para isso, utilizamos dados normalizados referentes a nutrientes, aplicamos técnicas de cálculo de similaridade por cosseno e construímos um modelo simples, porém eficaz, de recomendação baseado em conteúdo.

Este sistema não é personalizado, ou seja, não leva em conta preferências ou restrições individuais, mas estabelece uma base sólida para futuras melhorias e integrações com perfis de usuários.

As etapas principais envolvem:
- Preparação e tratamento dos dados nutricionais,
- Normalização dos valores para uniformizar escalas,
- Cálculo da similaridade vetorial entre alimentos,
- Seleção dos alimentos mais semelhantes para recomendação.
- Testes de métricas adequadas para Sistemas de Recomendação a partir de amostras obtidas na interface da aplicação do Streamlit.


# Limpeza e Tratamento dos dados

1. Aquisição dos dados
   Utilizamos o DataFrame df_insa contendo as informações nutricionais e categorias dos alimentos da Tabela INSA.

2. Tratamento dos dados
   - Valores ausentes nas colunas nutricionais foram substituídos por zero, considerando ausência de dado como ausência do nutriente.
   - Devido ao grande número de valores únicos nas colunas “Nível 1”, “Nível 2” e “Nível 3”,  optou-se por utilizar apenas os atributos numéricos da tabela para o cálculo da similaridade do cosseno, excluindo estas colunas.

3. Normalização dos dados
   - Aplicação do StandardScaler para padronizar (z-score) os dados nutricionais, garantindo que todos os atributos contribuam igualmente na similaridade.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/df_insa.zip /content/
!unzip -q /content/df_insa.zip

In [ ]:
# Carregar os dados
import pandas as pd

df_insa = pd.read_csv('df_insa.csv', header=0)
df_insa.head()

,Nome do alimento,Categorias do modelo,Quantidade de categorias,Nível 1,Nível 2,Nível 3,Energia\n[kcal],Energia\n[kJ],Lípidos\n[g],Ácidos gordos saturados\n[g],...,Cinza \n[g],Sódio \n[mg],Potássio \n[mg],Cálcio \n[mg],Fósforo \n[mg],Magnésio \n[mg],Ferro \n[mg],Zinco \n[mg],Selénio \n[µg],Iodo \n[µg]
0,Abacate,"Avocado 1, Avocado Black 1, Avocado Green 1, A...",4.0,Frutos e produtos derivados de frutos,Fruta utilizada como fruta,"Frutos diversos com casca não comestível, grandes",176,726,17.4,4.2,...,0.75,15.0,330.0,4.0,36.0,21.0,0.3,0.3,NaN,NaN
1,Abóbora cristalizada,NaN,NaN,Produtos hortícolas e derivados,Produtos hortícolas transformados ou em conser...,Produtos hortícolas cristalizados ou conservad...,293,1240,0.2,0.1,...,0.15,27.0,22.0,28.0,2.0,3.0,0.4,0.1,NaN,NaN
2,Abóbora crua,NaN,NaN,Produtos hortícolas e derivados,Frutos de hortícolas,Frutos vegetais de cucurbitáceas,11,47,0.2,0.1,...,0.40,1.0,200.0,25.0,5.0,5.0,0.1,0.1,NaN,NaN
3,Açafrão,NaN,NaN,"Leguminosas, frutos de casca rija, sementes ol...",Especiarias,"Flores ou partes de flores, utilizadas como es...",353,1490,5.9,1.6,...,3.00,150.0,1720.0,110.0,250.0,50.0,11.0,1.1,NaN,NaN
4,Açafrão-da-índia seco,NaN,NaN,"Leguminosas, frutos de casca rija, sementes ol...",Especiarias,Especiaria de de raízes e tubérculos,312,1300,7.0,2.9,...,7.08,31.0,2910.0,170.0,290.0,190.0,40.0,3.2,NaN,NaN


In [ ]:
df_insa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1330 entries, 0 to 1329
Data columns (total 48 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Nome do alimento                    1330 non-null   object 
 1   Categorias do modelo                63 non-null     object 
 2   Quantidade de categorias            63 non-null     float64
 3   Nível 1                             1330 non-null   object 
 4   Nível 2                             1330 non-null   object 
 5   Nível 3                             1329 non-null   object 
 6   Energia
[kcal]                      1330 non-null   int64  
 7   Energia
[kJ]                        1330 non-null   int64  
 8   Lípidos
[g]                         1330 non-null   float64
 9   Ácidos gordos saturados
[g]         1329 non-null   float64
 10  Ácidos gordos monoinsaturados 
[g]  1329 non-null   float64
 11  Ácidos gordos polinsaturados 
[g]   1330 no

In [ ]:
# Checar categorias únicas
df_insa['Nível 1 '].value_counts()

,count
Nível 1,
Carne e produtos cárneos,245
Pratos compostos,240
"Peixes, mariscos, anfíbios, répteis e invertebrados",144
Leite e produtos lácteos,123
Cereais e produtos à base de cereais,106
Produtos hortícolas e derivados,100
Frutos e produtos derivados de frutos,96
"Leguminosas, frutos de casca rija, sementes oleaginosas e especiarias",81
"Temperos, molhos e condimentos",34


In [ ]:
# Checar categorias únicas
df_insa['Nível 2'].value_counts()

,count
Nível 2,
"Pratos, incl. refeições prontas a comer (excluindo sopas e saladas)",195
Carne de mamiferos e de aves,175
Peixe (músculo),103
Fruta utilizada como fruta,53
Queijo,50
...,...
Produtos de raízes e tubérculos amiláceos,1
Extratos de origem vegetal,1
Sangue de animal,1


In [ ]:
# Checar categorias únicas
df_insa['Nível 3 '].value_counts()

,count
Nível 3,
"Pratos, excluindo pratos de massa ou arroz, sanduíches e pizza",153
Carne de mamíferos,131
Peixe de mar,95
Carne de aves,44
Sopas (prontas a comer),42
...,...
"Vieiras, pectens",1
Proteína de leite,1
Ostras,1


In [ ]:
#  Selecionar as colunas numéricas relevantes
category_cols = ['Nome do alimento', 'Nível 1 ', 'Nível 2', 'Nível 3 ', 'Categorias do modelo', 'Quantidade de categorias']

nutrient_df = df_insa.drop(columns=category_cols).copy()

# Tentar converter todas as colunas para float
nutrient_df = nutrient_df.apply(pd.to_numeric, errors='coerce')
nutrient_df.index = df_insa.index
nutrient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1330 entries, 0 to 1329
Data columns (total 42 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Energia
[kcal]                      1330 non-null   int64  
 1   Energia
[kJ]                        1330 non-null   int64  
 2   Lípidos
[g]                         1330 non-null   float64
 3   Ácidos gordos saturados
[g]         1329 non-null   float64
 4   Ácidos gordos monoinsaturados 
[g]  1329 non-null   float64
 5   Ácidos gordos polinsaturados 
[g]   1330 non-null   float64
 6   Ácido linoleico 
[g]                1330 non-null   float64
 7   Ácidos gordos trans 
[g]            1330 non-null   float64
 8   Hidratos de carbono 
[g]            1329 non-null   float64
 9   Açúcares 
[g]                       1330 non-null   float64
 10  Oligossacáridos 
[g]                1330 non-null   float64
 11  Amido 
[g]                          1330 no

In [ ]:
# Lidar com valores ausentes
nutrient_df = nutrient_df.fillna(0)

# Escalonar os dados (normalização)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
nutrient_scaled = scaler.fit_transform(nutrient_df)

# Recomendação baseada no cosine similarity

1. Cálculo da similaridade  
   - Utilização da função cosine_similarity do scikit-learn para calcular a similaridade do cosseno entre os vetores nutricionais normalizados.  
   - Geração de uma matriz quadrada que representa o grau de similaridade entre cada par de alimentos (valores entre -1 e 1).

2. Sistema de recomendação
   - Para um alimento dado (índice conhecido), identificam-se os 3 alimentos mais similares com base na matriz de similaridade.  
   - Exclui-se o próprio alimento da lista de recomendação para evitar redundância.

3. Persistência dos dados
   - A matriz de similaridade é salva em cosine_similarity.pkl para uso eficiente na aplicação.

In [ ]:
# Calcular similaridade com Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity

cos_sim = cosine_similarity(nutrient_scaled)

In [ ]:
cos_sim

array([[ 1.        , -0.06698718,  0.48216579, ..., -0.26029183,
        -0.20862449, -0.19466738],
       [-0.06698718,  1.        ,  0.03818531, ..., -0.1868427 ,
        -0.18265709, -0.1763667 ],
       [ 0.48216579,  0.03818531,  1.        , ..., -0.19140336,
        -0.20621276, -0.17223653],
       ...,
       [-0.26029183, -0.1868427 , -0.19140336, ...,  1.        ,
         0.95365965,  0.94489106],
       [-0.20862449, -0.18265709, -0.20621276, ...,  0.95365965,
         1.        ,  0.99281709],
       [-0.19466738, -0.1763667 , -0.17223653, ...,  0.94489106,
         0.99281709,  1.        ]])

In [ ]:
import numpy as np

def get_top_similar(index, sim_matrix, top_n=3):
    # Obter similaridades para um alimento (linha da matriz)
    similarities = sim_matrix[index]

    # Obter índices ordenados por similaridade decrescente, excluindo ele mesmo (index)
    similar_indices = np.argsort(similarities)[::-1]
    similar_indices = [i for i in similar_indices if i != index][:top_n]

    return df_insa.iloc[similar_indices]


In [ ]:
# Teste simples com alimento 0 "Abacate"
alimento_index = 1
top_similares_df = get_top_similar(alimento_index, cos_sim)

In [ ]:
top_similares_df

,Nome do alimento,Categorias do modelo,Quantidade de categorias,Nível 1,Nível 2,Nível 3,Energia\n[kcal],Energia\n[kJ],Lípidos\n[g],Ácidos gordos saturados\n[g],...,Cinza \n[g],Sódio \n[mg],Potássio \n[mg],Cálcio \n[mg],Fósforo \n[mg],Magnésio \n[mg],Ferro \n[mg],Zinco \n[mg],Selénio \n[µg],Iodo \n[µg]
266,Geleia de casca de laranja,NaN,NaN,Frutos e produtos derivados de frutos,Produtos de frutos processados,"Frutos / Vegetais, cremes e similares",275,1170,0.0,0.0,...,0.30,18.0,44.0,35.0,10.0,4.0,0.7,0.1,NaN,NaN
434,Pera cristalizada,NaN,NaN,Frutos e produtos derivados de frutos,Produtos de frutos processados,Outros frutos processados (excluindo bebidas),285,1210,0.1,0.0,...,0.15,65.0,17.0,13.0,4.0,5.0,0.3,0.2,NaN,NaN
360,Mel de cana,NaN,NaN,"Açúcar e similares, confeitaria e sobremesas d...",Açúcar e outros adoçantes (excluindo os adoçan...,Xaropes (melaços e outros xaropes),311,1320,0.0,0.0,...,0.39,270.0,58.0,16.0,1.0,3.0,0.4,0.1,NaN,NaN


In [ ]:
import pickle

# Salvar
with open("cosine_similarity.pkl", "wb") as f:
    pickle.dump(cos_sim, f)

In [ ]:
# Carregar
with open("cosine_similarity.pkl", "rb") as f:
    cos_sim = pickle.load(f)

In [ ]:
# !cp /content/cosine_similarity.pkl /content/drive/MyDrive/

# Testes dos Resultados na Aplicação do Streamlit

- Esta seção apresenta as e métricas dos resultados obtidos nos testes realizados na aplicação do NutriVisão no Streamlit.
- Foram coletadas para análise 5 informações nutricionais e recomandações de alimentos similares, baseado no modelo da similaridade do cosseno criado acima.
- Os alimentos são:  castanha (1 semente), cenoura e pepino (2 legumes) e pera e banana (2 frutas).
- Os dados foram obtidos a partir da interface da aplicação, através da funcionalidade do Streamlit de pode fazer download da tabela apresentada na página. Como são apresentadas de maneira separada as informações nutricionais do alimento detetado pelo modelo CNN e as informações dos alimentos similares, foi necessário fazer uma limpeza de dados antes do cálculo das métricas.
- As métricas escolhidas foram:
  - A. Acurácia por taxonomia (INSA):
    - precision@k: fração dos k recomendados que têm o mesmo nível.
    - recall@k: fração dos relevantes totais (no dataset) recuperada nos k.
    - NDCG@k: dá mais peso às posições de topo (ganho 1 se mesmo nível, 0 caso contrário).
  - B. Fidelidade nutricional (quão “parecido” é o perfil) Como a recomendação é baseada em nutrientes, meça a distância nutricional efetiva entre a consulta e os k itens retornados:
    - Δ% médio por nutriente: para cada nutriente padronizado (z-score), calcule |recomendado − consulta| e faça a média nos k itens.
    - RMSE de nutrientes (em z-score) entre consulta e cada recomendado; reporte a média dos k. Quanto menor, melhor.
    - Correlação de ranking de nutrientes (Spearman) entre consulta e recomendado: mede se os “nutrientes dominantes” batem.

In [ ]:
from sklearn.metrics import roc_auc_score
from scipy.stats import spearmanr
import glob
import numpy as np
import pandas as pd

from google.colab import files
uploaded = files.upload()

Saving teste_banana.csv to teste_banana.csv
Saving teste_banana_similares.csv to teste_banana_similares.csv
Saving teste_castanha.csv to teste_castanha.csv
Saving teste_castanha_similares.csv to teste_castanha_similares.csv
Saving teste_cenoura.csv to teste_cenoura.csv
Saving teste_cenoura_similares.csv to teste_cenoura_similares.csv
Saving teste_pepino.csv to teste_pepino.csv
Saving teste_pepino_similares.csv to teste_pepino_similares.csv
Saving teste_pera.csv to teste_pera.csv
Saving teste_pera_similares.csv to teste_pera_similares.csv


In [ ]:
alimentos = ["banana", "castanha", "cenoura", "pepino", "pera"]

for a in alimentos:
    df_base = pd.read_csv(f"teste_{a}.csv")
    df_sim = pd.read_csv(f"teste_{a}_similares.csv")

    # Cria variável dinamicamente: df_banana, df_castanha, etc.
    globals()[f"df_{a}"] = pd.concat([df_base, df_sim], ignore_index=True)

In [ ]:
dfs = [df_banana, df_castanha, df_cenoura, df_pepino, df_pera]

for df in dfs:
    if 'Unnamed: 0' in df.columns:
        df.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

In [ ]:
df_banana.head()

,id,Nome do alimento,Nível 1,Nível 2,Nível 3,Energia [kcal],Energia [kJ],Lípidos [g],Ácidos gordos saturados [g],Ácidos gordos monoinsaturados [g],...,Cinza [g],Sódio [mg],Potássio [mg],Cálcio [mg],Fósforo [mg],Magnésio [mg],Ferro [mg],Zinco [mg],Selénio [µg],Iodo [µg]
0,57,Banana,Frutos e produtos derivados de frutos,Fruta utilizada como fruta,"Frutos diversos com casca não comestível, grandes",104,441,0.4,0.1,0.0,...,0.89,6.0,430,8,25,28,0.4,0.2,NaN,3.8
1,0,Cebola frita com óleo alimentar,Produtos hortícolas e derivados,Bolbos,Cebolas e similares,138,573,11.2,1.3,2.3,...,1.40,40.0,420,62,60,24,1.0,0.6,NaN,1.7
2,1,"Ervilhas, grão, frescas cozidas","Leguminosas, frutos de casca rija, sementes ol...",Leguminosas,"Sementes de leguminosas frescas (feijão, ervil...",72,304,0.7,0.1,0.1,...,0.66,110.0,330,37,68,21,1.1,0.4,NaN,0.2
3,2,Anona,Frutos e produtos derivados de frutos,Fruta utilizada como fruta,"Frutos diversos com casca não comestível, grandes",82,349,0.4,0.0,0.2,...,0.55,11.0,240,6,31,23,0.3,0.2,NaN,NaN
4,3,Endívia crua,Produtos hortícolas e derivados,Hortícolas folhosos,Alfaces e outros vegetais para salada,19,80,0.4,0.1,0.0,...,0.60,22.0,380,44,36,10,2.8,0.2,1.0,6.4


In [ ]:
eps = 1e-8

# --- helpers métricas ---
def precision_at_k(truth, recs, k):
    return len(set(truth) & set(recs[:k])) / k

def recall_at_k(truth, recs, k):
    return len(set(truth) & set(recs[:k])) / len(truth) if truth else 0.0

def ndcg_at_k(truth, recs, k):
    gains = [1 if r in truth else 0 for r in recs[:k]]
    dcg = sum(g / np.log2(i + 2) for i, g in enumerate(gains))
    ideal = sorted(gains, reverse=True)
    idcg = sum(g / np.log2(i + 2) for i, g in enumerate(ideal))
    return dcg / idcg if idcg > 0 else 0.0

# --- função principal ---
def evaluate_combined_df(df, k_list=(1,3,5,10), categorical_cols=None):
    """
    df: DataFrame onde
        - linha 0 = alimento-consulta (query)
        - linhas 1: = similares (ordered)
    Retorna: meta(dict), metrics_by_k (DataFrame), per_similar (DataFrame)
    """
    # limpar nomes de colunas (remoção de espaços/tabs acidentais)
    df = df.copy()
    df.columns = df.columns.str.strip()

    # colunas categóricas padrão (ajusta se o teu CSV tiver nomes diferentes)
    if categorical_cols is None:
        categorical_cols = ["Nome do alimento", "Nível 1", "Nível 2", "Nível 3", "id", "tipo"]
    categorical_cols = [c for c in categorical_cols if c in df.columns]

    # query e similares
    query_row = df.iloc[0]
    sims_df = df.iloc[1:].reset_index(drop=True)
    qname = query_row["Nome do alimento"]

    # identifica colunas de nutrientes (todas exceto as categóricas)
    nutrient_cols = [c for c in df.columns if c not in categorical_cols]
    if len(nutrient_cols) == 0:
        # fallback: tenta detectar colunas numéricas
        nutrient_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    # coerção para numérico (evita strings)
    qvec = pd.to_numeric(query_row[nutrient_cols], errors='coerce').fillna(0).to_numpy(dtype=float)
    rec_vecs = sims_df[nutrient_cols].apply(pd.to_numeric, errors='coerce').fillna(0).to_numpy(dtype=float)
    rec_names = sims_df["Nome do alimento"].astype(str).tolist()

    # relevantes DENTRO da lista de similars (os que partilham o mesmo Nível 3 que o query)
    qlevel3 = query_row.get("Nível 3", None)
    if qlevel3 is None:
        relevant_list = []
    else:
        relevant_list = sims_df.loc[sims_df["Nível 3"] == qlevel3, "Nome do alimento"].astype(str).tolist()

    # métricas por k (inclui também medidas de fidelidade agregadas ao top-k)
    rows_k = []
    for k in k_list:
        prec = precision_at_k(relevant_list, rec_names, k)
        rec = recall_at_k(relevant_list, rec_names, k)
        ndcg = ndcg_at_k(relevant_list, rec_names, k)

        topk_vecs = rec_vecs[:k]
        if topk_vecs.shape[0] > 0:
            # Δ% médio sobre nutrientes para cada similar no top-k, depois média
            delta_topk = np.mean(np.mean(np.abs(topk_vecs - qvec) / (np.abs(qvec) + eps), axis=1))
            rmse_topk  = np.mean(np.sqrt(np.mean((topk_vecs - qvec)**2, axis=1)))
            spears = []
            for rvec in topk_vecs:
                try:
                    s, _ = spearmanr(qvec, rvec)
                except Exception:
                    s = np.nan
                spears.append(s)
            spear_topk = np.nanmean(spears)
        else:
            delta_topk = np.nan; rmse_topk = np.nan; spear_topk = np.nan

        rows_k.append({
            "k": k,
            "precision": prec,
            "recall": rec,
            "ndcg": ndcg,
            "delta_topk_mean": delta_topk,
            "rmse_topk_mean": rmse_topk,
            "spearman_topk_mean": spear_topk
        })
    metrics_by_k = pd.DataFrame(rows_k)

    # detalhes por similar (Δ%, RMSE, Spearman) para todos os similares na lista
    details = []
    for i, name in enumerate(rec_names):
        rvec = rec_vecs[i]
        delta = np.abs(rvec - qvec) / (np.maximum(np.abs(qvec), np.abs(rvec)) + 1e-8)
        delta_pct = np.mean(delta)
        rmse = float(np.sqrt(np.mean((rvec - qvec)**2)))
        try:
            s, _ = spearmanr(qvec, rvec)
        except Exception:
            s = np.nan
        details.append({"similar": name, "rank_pos": i+1, "delta_pct": delta_pct, "rmse": rmse, "spearman": s,
                        "same_level3": (sims_df.loc[i, "Nível 3"] == qlevel3)})
    per_similar = pd.DataFrame(details)

    meta = {
        "query_name": qname,
        "n_similars": len(rec_names),
        "n_relevant_in_list": len(relevant_list),
        "nutrient_cols_used": nutrient_cols
    }
    return meta, metrics_by_k, per_similar

In [ ]:
for name in ["banana","castanha","cenoura","pepino","pera"]:
    df = globals().get(f"df_{name}")
    if df is None:
        continue
    meta, metrics_k, details = evaluate_combined_df(df)
    print(f"\n== {meta['query_name']} ==")
    print(metrics_k)


== Banana ==
    k  precision  recall  ndcg  delta_topk_mean  rmse_topk_mean  \
0   1   0.000000     0.0   0.0     2.714286e+07       24.538135   
1   3   0.333333     1.0   0.5     1.095238e+07       35.922352   
2   5   0.200000     1.0   0.5     8.804764e+06       49.096319   
3  10   0.100000     1.0   0.5     4.783335e+06       47.802500   

   spearman_topk_mean  
0            0.647346  
1            0.809809  
2            0.823872  
3            0.838540  

== Castanha, miolo ==
    k  precision  recall  ndcg  delta_topk_mean  rmse_topk_mean  \
0   1   1.000000     1.0   1.0     6.590684e-02       15.148084   
1   3   0.333333     1.0   1.0     2.380953e+06       47.937810   
2   5   0.200000     1.0   1.0     2.476191e+06       82.517762   
3  10   0.100000     1.0   1.0     1.282619e+07      102.107780   

   spearman_topk_mean  
0            0.999466  
1            0.903158  
2            0.823343  
3            0.784062  

== Cenoura crua ==
    k  precision  recall      n

1. Banana
- Precision cai de 0.33 (k=3) para 0.1 (k=10) → no top-10, poucos itens são realmente relevantes.
- Recall já bate 1.0 a partir de k=3 → ou seja, com até 3 vizinhos já recupera todos os similares esperados.
- nDCG = 0.5 em k=3 e k=10 → os relevantes não estão nas primeiras posições, mas aparecem.
- RMSE cresce com k → os vizinhos mais distantes nutricionalmente entram no ranking.
- Spearman ~0.83 em k=5–10 → boa correlação no perfil nutricional médio.
- O modelo consegue recuperar os similares da banana, mas mistura ruído em k altos.

2. Castanha
- Precision@1 = 1.0 → o 1º vizinho é perfeito.
- Recall@3+ = 1.0 → encontra todos os relevantes.
- nDCG ~1.0 → os relevantes estão bem ranqueados.
- RMSE sobe muito em k=10 (mais ruído).
- Spearman ~0.99 em k=1 → quase idêntico nutricionalmente.
- Ótimas recomendações.

3. Cenoura
- Precision@1 = 0 → não recupera similar logo no topo.
- Só em k=10 recall = 1.0 → os similares aparecem apenas bem abaixo no ranking.
- nDCG baixo (0.31 em k=10) → ranking ruim, relevantes jogados para o fim.
- RMSE muito alto (>400 kcal/unidade dos vetores) → os vizinhos têm perfis bem diferentes.
- Spearman alto (~0.88) → apesar da magnitude diferente, a tendência relativa de nutrientes é parecida.
- O modelo tem dificuldade em recomendar similares diretos para cenoura. Talvez falte densidade de dados ou categorias fortes para esse alimento.

4. Pepino cru
- Precision@1 = 0, melhora só em k=10 (0.1).
- Recall = 1.0 apenas em k=10.
- nDCG baixo (0.28 em k=10) → ranking fraco.
- RMSE relativamente baixo (≈20–27) → diferença nutricional não é absurda.
- Spearman até 0.80 → o padrão de nutrientes é bem correlacionado.
- O pepino é nutricionalmente próximo dos vizinhos, mas o ranking não prioriza bem os similares verdadeiros.

5. Pera
- Precision@1 = 1.0 e recall crescente até 1.0 em k=10 → excelentes recomendações.
- nDCG >0.97 em todos os k → ranking muito bom.
- RMSE baixo (7–15) → vizinhos realmente parecidos nutricionalmente.
- Spearman >0.90 (exceto k=10) → padrão nutricional quase idêntico.
- O modelo lida muito bem com a família da pera, captando as variedades.

6. Geral
- Os resultados mostram que, mesmo sem considerar níveis taxonômicos, o modelo consegue identificar vizinhos com perfis nutricionais semelhantes, refletido em Spearman_topk_mean alto, enquanto Precision e Recall aumentam conforme expandimos k, mostrando uma boa recuperação de substitutos nutricionalmente relevantes.